In [30]:
import csv
import os

import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

We define a function which takes the signals and extracts relevant features:

In [31]:

def get_features(sample_data):  
    signal_data = []
    for i in range(len(sample_data)):   
            signal = sample_data[i]

            #initialize variables 
            mean_dist, median_dist, var_dist, max_dist, min_dist = -1000, -1000, -1000, -1000, -1000
            mean_r_amp, median_r_amp, var_r_amp, min_r_amp, max_r_amp, = -1000, -1000, -1000, -1000, -1000
            mean_max, median_max, var_max, min_max, max_max = -1000, -1000, -1000, -1000, -1000
            mean_min, median_min, var_min, min_min, max_min = -1000, -1000, -1000, -1000, -1000
            min_var, max_var, mean_var, median_var, var_var = -1000, -1000, -1000, -1000, -1000


            r_peaks = ecg.engzee_segmenter(signal, sampling_rate=200)['rpeaks']
            num_peaks = len(r_peaks)
            min_sig = np.min(signal)
            max_sig = np.max(signal)
            var_sig = np.std(signal)
            mean_sig = np.mean(signal)
            median_sig = np.median(signal)

        
            if(num_peaks>1):
                #info  on distance between r peaks
                dist = np.diff(r_peaks)
                mean_dist = np.mean(dist)
                median_dist = np.median(dist)
                var_dist = np.std(dist)
                max_dist = np.max(dist)
                min_dist = np.min(dist)

                #info about r peak amplitude
                signal_at_r_peaks = signal[r_peaks]
                mean_r_amp = np.mean(signal_at_r_peaks)
                median_r_amp = np.median(signal_at_r_peaks)
                var_r_amp = np.std(signal_at_r_peaks)
                min_r_amp = np.min(signal_at_r_peaks)
                max_r_amp = np.max(signal_at_r_peaks)

            #seperate signal into heartbeats
            beats = ecg.extract_heartbeats(signal, r_peaks, 200)['templates']
            if(len(beats>2)):
                # info about the max of each beat, over all beats of the signal (should be equal ro r peaks)
                max_beats = np.max(beats, axis=1)
                mean_max = np.mean(max_beats)
                median_max = np.median(max_beats)
                var_max = np.std(max_beats)
                min_max = np.min(max_beats)
                max_max = np.min(max_beats)

                # info about the minimum of each beat, over all beats of the signal
                min_beats = np.min(beats, axis=1)
                mean_min = np.mean(min_beats)
                median_min = np.median(min_beats)
                var_min = np.std(min_beats)
                min_min = np.min(min_beats)
                max_min = np.min(min_beats)

            #info bout the variance between beats
                var_beats = np.std(beats, axis=0)
                min_var = np.min(var_beats)
                max_var = np.max(var_beats)
                mean_var = np.mean(var_beats)
                median_var = np.median(var_beats)
                var_var = np.std(var_beats)

            temp =  [
                num_peaks, 
                min_sig, max_sig, var_sig, mean_sig, median_sig,
                mean_dist, median_dist, var_dist, max_dist, min_dist,
                mean_r_amp, median_r_amp, var_r_amp, min_r_amp, max_r_amp,
                mean_max, median_max, var_max, min_max, max_max,
                mean_min, median_min, var_min, min_min, max_min,
                min_var, max_var, mean_var, median_var, var_var
            ]
            signal_data.append(temp)
            if i%200 == 0: print(i)
            if i%1000 == 0: print(temp)
    return signal_data



We Read in the training and test data. We crop the Signals NaN values and seperate the labels for the training set

In [33]:
#load data from csv
data_train = pd.read_csv('data/train.csv', index_col='id')
data_test = pd.read_csv('data/test.csv', index_col='id')

# Convert non-numeric values to NaN and drop them
data_train = data_train.apply(pd.to_numeric, errors='coerce')
data_test = data_test.apply(pd.to_numeric, errors='coerce')

labels_array = data_train['y'].to_numpy()

sample_data = []
for i in range(len(data_train)):
    sample_data.append(data_train.loc[i].dropna().to_numpy(dtype='float32'))
    if i % 1000 == 0: print(len(sample_data[i]))

test_data = []
for i in range(len(data_test)):
    test_data.append(data_test.loc[i].dropna().to_numpy(dtype='float32'))
    if i % 1000 == 0: print(len(test_data[i]))

16323
8781
10004


KeyboardInterrupt: 

In [7]:
X = get_features(sample_data)
X_test = get_features(test_data)

X_train, X_val, y_train, y_val = train_test_split(X, labels_array, test_size=0.1, random_state=42)


0
[66, -236.0, 738.0, 108.992065, 9.163756, -22.0, 247.03076923076924, 249.0, 6.560884627898262, 256, 227, 504.19696, 494.0, 72.27938, 343.0, 738.0, 504.66153, 494.0, 72.735405, 343.0, 343.0, -99.61539, -97.0, 38.534576, -236.0, -236.0, 37.570152, 78.569244, 49.851723, 46.697144, 10.19797]
0
[32, -774.0, 1713.0, 295.1696, 33.81703, 8.0, 257.258064516129, 252.0, 17.417024933788916, 292, 237, 1315.5625, 1310.5, 248.37785, 617.0, 1713.0, 1339.0968, 1323.0, 216.26842, 791.0, 791.0, -226.19354, -168.0, 198.2378, -693.0, -693.0, 158.15321, 272.8997, 221.13374, 215.78296, 25.27887]


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Conv1D, Flatten

# Convert labels to categorical one-hot encoding
y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_val)

X_train = np.array(X_train)
X_val = np.array(X_val)



# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Compile the model
# Adjust the learning rate
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', AUC(name='f1_score')])

# Train the model
model.fit(X_train, y_train_categorical, epochs=500, batch_size=64)

# Evaluate the model
y_pred_train = model.predict(X_train)
y_pred_train_classes = np.argmax(y_pred_train, axis=1)
f1_train = f1_score(y_train, y_pred_train_classes, average='micro')

y_pred_val = model.predict(X_val)
y_pred_val_classes = np.argmax(y_pred_val, axis=1)
f1_val = f1_score(y_val, y_pred_val_classes, average='micro')
print('f1 train score: ', f1_train)
print('f1 val score: ', f1_val)


Epoch 1/500


c:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3056 - f1_score: 0.5095 - loss: 1.4200
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3056 - f1_score: 0.5158 - loss: 1.4155
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3056 - f1_score: 0.5082 - loss: 1.4111
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3056 - f1_score: 0.5204 - loss: 1.4067
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3056 - f1_score: 0.5170 - loss: 1.4025
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3056 - f1_score: 0.5229 - loss: 1.3983
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3056 - f1_score: 0.5069 - loss: 1.3941
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3056 - f1_score: 0.5288 - loss: 1.3900
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3056 - f1_score: 0.5125 - loss: 1.3860
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3056 - f1_score: 0.525

In [ ]:
labels_categorical = to_categorical(labels_array)
X = np.array(X)

#model.fit(X, labels_categorical, epochs=50, batch_size=64)
y_pred_test = model.predict(np.array(X_test))
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

table = pd.DataFrame({'id': np.arange(0, y_pred_test_classes.shape[0]), 'y': y_pred_test_classes.flatten()})
table.to_csv("./data/y_test_pred.csv", index=False)

107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step
